# Introduction to LangChain LCEL
### LangChain Expression Language (LCEL)

Alejandro Ricciardi (Omegapy)  
created date: 01/10/2024   
[GitHub](https://github.com/Omegapy)  

Credit: [LangChain](https://python.langchain.com/docs/expression_language/) 

Projects Description:  
**LangChain** is a framework for developing applications powered by language models. It enables applications that:

**In this project:**  I get started using LCEL.  

<p></p>
<b class="alert alert-block alert-info" style="font-size:15;">
⚠️ This project requires an OpenAi key.
</b>


Project map:
- API Key
- Getting started
    - Basic example **```chain = prompt | model | output_parser```**
        - Prompt (Human Message) **```prompt_value = prompt.invoke({"topic": "ice cream"})```**
        - Model (AIMessage) **```message = model.invoke(prompt_value)```**
                - ChatModel Output
                - LLM Output
        - Output parser (takes AIMessage or any string and converts it) **```output_parser.invoke(message)```**
        - Entire Pipeline **```(prompt | model | output_parser).invoke({"topic": "ice cream"})```**
- RAG Search **```chain = setup_and_retrieval | prompt | model | output_parser```**
    - Example
    - Example Explanation


##### API Keys

In [1]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.environ.get("OPEN_AI_KEY")

## Getting started

### Basic example: 
prompt + model + output parser

In [2]:
from langchain_core.output_parsers import StrOutputParser #https://python.langchain.com/docs/modules/model_io/output_parsers/
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [3]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

"Why don't ice creams ever get invited to parties?\n\nBecause they always melt under pressure!"

 #### Prompt (Human Message)
```prompt``` is a ```BasePromptTemplate```, which means it takes in a dictionary of template variables and produces a ```PromptValue```.   

A ```PromptValue``` is a wrapper around a completed ```prompt``` that can be passed to either an ```LLM``` (which takes a string as input) or ```ChatModel``` (which takes a sequence of messages as input). 
It can work with either language model type because it defines logic both for producing BaseMessages and for producing a string.

In [4]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [5]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [6]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

#### Model (AIMessage)
The ```PromptValue``` is then passed to ```model```. In this case our ```model``` is a ```ChatModel```, meaning it will output a ```BaseMessage```.

##### ChatModel Output

In [7]:
message = model.invoke(prompt_value)
message

AIMessage(content="Why don't ice creams ever get invited to parties?\n\nBecause they always drip when they're getting ready!")

##### LLM Output

In [8]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nRobot: Why did the ice cream go to therapy? Because it was feeling a little melted.'

#### Output parser (takes AIMessage or any string and converts it)
And lastly we pass our ```model``` output to the ```output_parser```, which is a ```BaseOutputParser``` meaning it takes either a ```string``` or a ```BaseMessage``` as input. 
The ```StrOutputParser``` specifically simple converts any input into a ```string```.

In [9]:
output_parser.invoke(message)

"Why don't ice creams ever get invited to parties?\n\nBecause they always drip when they're getting ready!"

#### Entire Pipeline
**```(prompt | model | output_parser).invoke({"topic": "ice cream"})```**

To follow the steps along:
1. We pass in user input on the desired topic as ```{"topic": "ice cream"}```
2. The ```prompt``` component takes the user input, which is then used to construct a ```PromptValue``` after using the ```topic``` to construct the prompt.
3. The ```model``` component takes the generated ```prompt```, and passes into the ```OpenAI LLM model``` for evaluation. The generated output from the model is a ```ChatMessage``` object.
4. Finally, the ```output_parser``` component takes in a ```ChatMessage```, and transforms this into a Python ```string```, which is returned from the ```invoke``` method.

Note that if you’re curious about the output of any components, you can always test out a smaller version of the chain such as prompt or prompt | model to see the intermediate results:



In [10]:
chain = prompt | model | output_parser

(prompt | model | output_parser).invoke({"topic": "ice cream"})

"Why don't ice creams ever get invited to parties?\n\nBecause they always bring a meltdown!"

## RAG Search

### Example
For our next example, we want to run a retrieval-augmented generation chain to add some context when responding to questions.

To be error free this example requires:
```
pydantic==1.10.8 
docarray==0.40.0 
langchain docarray tiktoken
```

In [11]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

Embedding (OpenAI) - Vectorstore

In [12]:
vectorstore = DocArrayInMemorySearch.from_texts( # Text Doc in memory 
    ["harrison worked at kensho", "bears like to eat honey"], # Doc-text
    embedding=OpenAIEmbeddings(), # embedding
)
retriever = vectorstore.as_retriever()

In [13]:
template = """
Answer the question based only on the following context: {context}

Question: {question}
"""

In [14]:
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

##### RunnableParallel object

In [15]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

In [21]:
chain = setup_and_retrieval | prompt | model | output_parser
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

### Example Explanation

To explain this, we first can see that the prompt template above takes in ``context`` and ```question``` as values to be substituted in the prompt. Before building the prompt template, we want to retrieve relevant documents to the search and include them as part of the context. 
**```setup_and_retrieval = RunnableParallel( {"context": retriever, "question": RunnablePassthrough()} )```**

As a preliminary step, we’ve setup the retriever using an in memory store, which can retrieve documents based on a query. This is a runnable component as well that can be chained together with other components, but you can also try to run it separately:

In [18]:
retriever.invoke("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [20]:
setup_and_retrieval

{
  context: VectorStoreRetriever(tags=['DocArrayInMemorySearch'], vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x000002357F3A55D0>),
  question: RunnablePassthrough()
}

With the flow being:

1. The first steps create a ```RunnableParallel``` object with two entries. 
- The first entry, ```context``` will include the document results fetched by the ```retriever```. 
- The second entry, question will contain the user’s original ```question```. To pass on the question, we use ```RunnablePassthrough``` to copy this entry.
2. Feed the dictionary from the step above to the ```prompt``` component. It then takes the user input which is ```question``` as well as the ```retrieved document``` which is ```context``` to construct a prompt and output a ```PromptValue```.
3. The ```model``` component takes the generated prompt, and passes into the ```OpenAI LLM``` model for evaluation. The generated output from the model is a ```ChatMessage``` object.
4. Finally, the ```output_parser``` component takes in a ```ChatMessage```, and transforms this into a Python ```string```, which is returned from the invoke method.